## Quantum Multiplier   
The aim of this exercise is to design a quantum multiplier, which takes two integers _(neccessarily of the same bit length)_ as input and gives back their product as an integer. We will be designing a qauntum equivalent of the **shift and add multiplication algorithm** with optimal number of T-gates.  
  
  
Quantum circuits for integer arithmetic operations such as addition, subtraction and multiplication are important as they are required in the quantum circuit implementations of many quantum algorithms in these areas. Traditionally, many algorithms have been proposed for a quantum multiplier based on **Quantum Fourier Transform**, with many improvements in terms of quantum gates, number of qubits and delay. However most of them fail to reduce garbage outputs and **T-count** of the qubits.  

Fault tolerant gates, such as the set of Clifford + T gates are being used for making computers error prone. But, the increased tolerance to noise errors comes with increased implementation overhead associated with them. The T gates are commomnly used for designing algorithms and the T-count is calculated to have a measure of their performance. 

$$T = \begin{bmatrix} 1 & 0 \\ 0 & e^{i\pi/2} \end{bmatrix}$$  

Methods based on QFT have achieved garbageless circuit designs but have significant Clifford + T-gate costs.  

![T-Gate Cost](img1.png)

Here, we discuss the method proposed by **Edgard and Himanshu**, which only utilises the CNOT and Toffoli gates (T-count = 15). In this method, we use **Benett's Garbage Removal Scheme** to to remove the garbage outputs and either restore them to their original values or to the zero state. It involves applying the inverse of the multiplier operator and CNOT to restore the values of input numbers and to find their product. The scheme is depicted in the below given picture. 

![Bennets](img2.png)
  

The Quantum Shift and Add multiplication is constituted of Quantum Conditional Addition which will be discussed later in this exercise. 


In [1]:
import numpy as np
from qiskit import *
import time

In [2]:
a = int((input("Enter number 1 : ")))
b = int((input("Enter number 2 : ")))

Enter number 1 : 1152921504606846975
Enter number 2 : 1152921504606846975


We first convert the input numbers into their corresponding binary value and then store the digits in their respective lists. 

In [3]:
start = time.time()

bin_a = bin(a)[2:]
bin_b = bin(b)[2:]

print("binary of a : ", bin_a)
print("binary of b : ", bin_b)

a_index = [int(x) for x in [*bin_a]]
b_index = [int(x) for x in [*bin_b]]



a_index.reverse()
b_index.reverse()

""" Uncomment this part to see how if the numbers are not of the same bit length, the algorithm does not give a 
correct solution even if we add 0 bits to make them of the same size. """

# if len(a_index) > len(b_index) : 
#     b_index += [0] * (len(a_index) - len(b_index))
# elif len(a_index) < len(b_index) : 
#     a_index += [0] * (len(b_index) - len(a_index))
# else : 
#     pass

print("Index list for binary a, starting from 0th index : ", a_index)
print("Index list for binary b, starting from 0th index : ", b_index)

binary of a :  111111111111111111111111111111111111111111111111111111111111
binary of b :  111111111111111111111111111111111111111111111111111111111111
Index list for binary a, starting from 0th index :  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Index list for binary b, starting from 0th index :  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


## Quantum Conditional Addition  

In this part we design the ctrl_add circuit with no input carry. This design has no garbage outputs and improves the T-count as compared to the other multiplier algorithms. The circuit takes $a_{i}$ , $b_{i}$ , a control bit and two extra bits as an input. At the end of the computation, it restores the binary of number a, gives the binary of the sum of the two numbers on the register B and on one of the extra registers and restores the other extra register to zero state. The sample circuit design of addition of 4 bit numbers is given below. 

![ctrl_add](img3.png) 

The algorithm for creating a generalised ctrl-add circuit using CNOT and Toffoli gates is programmed below. This function will be used as a sub-circuit/gate in the final Quantum Multiplier algorithm. 

In [4]:
def ctrl_circ(n_add) : 

    Ctrl = QuantumRegister(1)
    A = QuantumRegister(n_add)
    B = QuantumRegister(n_add)
    A_extra = QuantumRegister(2)
    
    ctrl_add = QuantumCircuit(Ctrl, A, B, A_extra)

    
    #Step 1
    for i in range(1,n_add) : 
        #print(i)
        ctrl_add.cx(A[i], B[i])

    #Step 2
    ctrl_add.ccx(Ctrl, A[n_add - 1], A_extra[0])
    for i in range(n_add - 2, 0, -1) : 
        #print("i",i)
        ctrl_add.cx(A[i], A[i+1])

    #Step 3
    for i in range(n_add-1) : 
        ctrl_add.ccx(B[i], A[i], A[i+1])

    #Step 4 
    ctrl_add.ccx(B[n_add-1],A[n_add-1],A_extra[1])
    ctrl_add.ccx(Ctrl, A_extra[1], A_extra[0])
    ctrl_add.ccx(B[n_add-1],A[n_add-1],A_extra[1])
    ctrl_add.ccx(Ctrl, A[n_add-1], B[n_add-1])


    #Step 5 
    for i in range(n_add-2,-1, -1) :
        ctrl_add.ccx(B[i], A[i], A[i+1])
        ctrl_add.ccx(Ctrl, A[i], B[i])

        
    #Step 6 
    for i in range(1, n_add - 1) :
        ctrl_add.cx(A[i], A[i+1])

        
    #Step 7 
    for i in range(1, n_add) : 
        ctrl_add.cx(A[i], B[i])

 
    ctrl_gate = ctrl_add.to_gate()
    return ctrl_gate
    

## Quantum Integer Multiplication Circuit  

In this part we program the multiplier circuit using toffoli gate arrays and the ctrl_add circuit defined before. We take the binary of the two numbers a and b, product register, P of length 2n and an extra register as input. After the implementation of the circuit, the number a and b are restored, and the extra register is restored to zero state. The sample circuit for 4 bit numbers is given below.   
![multiplier](img4.png) 

In [5]:
bin_a = bin(a)[2:]
bin_b = bin(b)[2:]

print("binary of a : ", bin_a)
print("binary of b : ", bin_b)


if len(bin_a) == len(bin_b) : 
    n = len(bin_a)
    a_list = [int(x) for x in [*bin_a]]
    a_list.reverse()
    b_list = [int(x) for x in [*bin_b]]
    b_list.reverse()
    a_extra_list = [0,0]
    p_list = [0] * 2 * n
    
else : 
    print("Numbers should be of same bit size")
    
# print(p_list)
print("indices of binary a from 0 to n-1 : ", a_list)
print("indices of binary b from 0 to n-1 : ", b_list)

binary of a :  111111111111111111111111111111111111111111111111111111111111
binary of b :  111111111111111111111111111111111111111111111111111111111111
indices of binary a from 0 to n-1 :  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
indices of binary b from 0 to n-1 :  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [6]:
# quantum registers

A = QuantumRegister(len(bin_a),'a')
B = QuantumRegister(len(bin_b),'b')
P = QuantumRegister(2*n + 1,'p')

#initializing the circuit with a_i, b_i
initial_vec = b_list + a_list + p_list
print("Initial input vector:", initial_vec)
multiplier = QuantumCircuit(B, A, P)

for i in range(len(initial_vec)) : 
    if initial_vec[i] == 1 : 
        multiplier.x(i)

print("Initial circuit with input state:")
print(multiplier)
print("---------------------------------------------------------------------------------------------")


#Step 1
for i in range(n) : 
    multiplier.ccx(B[0], A[i], P[i])

#Step 2
index_ctrl = [1]
index1 = list(range(n,2*n ))
index2 = list(range(2*n + 1, 3*n + 3))

all_index = index_ctrl+index1+index2

multiplier.append(ctrl_circ(n),all_index)


#Step 3 
for j in range(2,n) : 

    index_ctrl = [j]
    index1 = list(range(n, 2*n))

    index2 = list(range(2*n + j , 3*n + j + 2 ))
    all_index = index_ctrl+index1+index2
    #print(index1,index2, all_index)

    multiplier.append(ctrl_circ(n),all_index)

print(multiplier)


Initial input vector: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Initial circuit with input state:
       ┌───┐
  b_0: ┤ X ├
       ├───┤
  b_1: ┤ X ├
       ├───┤
  b_2: ┤ X ├
       ├───┤
  b_3: ┤ X ├
       ├───┤
  b_4: ┤ X ├
       ├───┤
  b_5: ┤ X ├
       ├───┤
  b_6: ┤ X ├
       ├───┤
  b_7: ┤ X ├
       ├───┤
  

Here, we measure all the qubits of the circuit and look at the output value using the qasm_simulator provided by qiskit. 

In [7]:
multiplier.measure_all()

""" Uncomment this part to see how the qasm_simulator performs"""

# sim = Aer.get_backend('qasm_simulator')
# result = execute(multiplier, backend = sim , shots = 1).result()
# counts = result.get_counts()
# print(counts)

sim = Aer.get_backend('aer_simulator_matrix_product_state')
multiplier = transpile(multiplier, sim)

# Run and get counts
result = sim.run(multiplier, shots = 1).result()
counts = result.get_counts(multiplier)
print(counts)
#plot_histogram(counts, title='Bell-State counts')

{'0111111111111111111111111111111111111111111111111111111111110000000000000000000000000000000000000000000000000000000000001111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111': 1}


Now, we store this result in a list and check the results. 

In [8]:
res_bin = list(counts.keys())[0]
print("Binary of the resultant vector : ", res_bin)
res_list = [int(x) for x in [*res_bin]]
print(res_list)


Binary of the resultant vector :  0111111111111111111111111111111111111111111111111111111111110000000000000000000000000000000000000000000000000000000000001111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [9]:
b_res = res_list[-1:-n-1:-1]
print(b_res)

a_res = res_list[-n-1:-2*n-1:-1]
print(a_res) 

if a_res == a_index and b_res == b_index : 
    print("A and B vector are restored")
    
prod_res = res_list[-2*n-1 : -4*n - 1 : -1]
print(prod_res)
prod = 0
for i in range(len(prod_res)) : 
    #print("i", i)
    prod += 2 ** i * prod_res[i]
    #print("prod", prod)
print(prod)
print("a", type(a), type(b))
if prod == a * b : 
    print("Hurray! The product is succefully calculated to be : ", prod)
else : 
    print("Error in calculating the product")
    
end = time.time()

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
A and B vector are restored
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
1329227995784915870597964051066650625
a <class 'int'> <class 'int'>
Hurray! The product is succefully calculated to be :  1329227995784915870597964051066650625


We see that the quantum register A and B have restored the value of the binary of the the input integers. It has also succefully calculated the product of the two numbers on the Product register, P, of the quantum circuit. 

In [10]:
elapsed_time = end - start
print('Execution time:', elapsed_time, 'seconds')

Execution time: 14.060560703277588 seconds


## Discussion  
- It should be noted that the algorithm does not work if the numbers input are of different **bit size**. It does not work even if we make the two numbers of same bit size by adding zeroes. This method requires the binary of the number to be in the most simplified form.  
- The capability of algorithm is also dependent on the simulator it is being run on. The **qasm_simulator** is capable of simulating lesser number of qubits and thus can only give product for number of maximum **bit size 9**. The **Matrix product state simulator** on the other hand is much more efficient and can calculate the product of numbers of bit size as large as **59**. The highest number it can deal with is **1152921504606846975** and can calculate it's square very quickly.  
- The algorithm is very fast and takes a maximum of 15 seconds to calculate the product. Product of number of smaller sizes can be calculated in as less as 0.7 seconds.  
- The proposed algorithm performs appreciably better than the other proposed algorithms with :  
T count = $ 21n^{2} - 14 $  
number of qubits = $ 4n + 1 $  
number of ancillae = $ 2n + 1$  
This shows much improvement than the previously proposed algorithms for a quantum multiplier.  
- The circuit has a depth of $3n + 1$, as seen from the circuit.  
- Considering the resources required for and the efficiency of the multiplier, the algorithm has the potential to show good results and may be run on the IBM Quantum computers to study the error tolerance.  

## References  
https://arxiv.org/abs/1706.05113  
https://qiskit.org/  
https://arxiv.org/abs/quant-ph/0008033  
https://arxiv.org/abs/2005.00443